In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
import urllib.request
import re
from bs4 import BeautifulSoup
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
BeautifulSoup	 DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 attempt_to_pickle	 example_iterrows	 
get_column_descriptions	 load_csv	 load_dataframes	 load_object	 math	 os	 pd	 pickle	 re	 
save_dataframes	 store_objects	 sys	 urllib	 


In [2]:

index_name = 'Unit Name'
dataframes_dict = load_dataframes(unit_availability_df='unit_availability_df')
unit_availability_df = dataframes_dict['unit_availability_df']
if 'Unnamed: 0' in unit_availability_df.columns:
    unit_availability_df = unit_availability_df.set_index('Unnamed: 0')
elif index_name in unit_availability_df.columns:
    unit_availability_df = unit_availability_df.set_index(index_name)
unit_availability_df.index.name = index_name

Attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\pickle\unit_availability_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\unit_availability_df.csv.



# Clean up attack columns

In [3]:

wiki_url = 'https://ageofempires.fandom.com'
def get_link_html(link):
    link_str = re.sub(' +', '_', link)
    link_url = '{}/wiki/{}'.format(wiki_url, link_str)
    try:
        extended_link_url = '{}_(Age_of_Empires_II)'.format(link_url)
        with urllib.request.urlopen(extended_link_url) as response:
            link_html = response.read()
        link_url = extended_link_url
    except:
        try:
            with urllib.request.urlopen(link_url) as response:
                link_html = response.read()
        except Exception as e:
            print('{} got this error: {}'.format(link_url, e))
            link_html = None
    
    return link_url, link_html

In [13]:

def get_page_soup(tech_name):
    page_url, page_html = get_link_html(tech_name)
    try:
        page_soup = BeautifulSoup(page_html, 'html.parser')
    except Exception as e:
        print('{} got this error: {}'.format(page_url, e))
        page_soup = None
    
    return page_soup

In [14]:

content_text_selector = '#mw-content-text'
def get_content_text_div(tech_name):
    page_soup = get_page_soup(tech_name)
    try:
        content_text_div = page_soup.select(content_text_selector)[0]
    except Exception as e:
        print('{} got this error: {}'.format(tech_name, e))
        content_text_div = None
    
    return content_text_div

In [11]:

def get_attack_from_aside(unit_soup):
    attack_value = None
    aside_list = unit_soup.select('aside')
    for aside_soup in aside_list:
        for data_source_div in aside_soup.find_all('div', {'data-source': True}):
            data_source_h3_list = data_source_div.find_all('h3')
            if len(data_source_h3_list):
                h3_siblings_list = list(data_source_h3_list[0].next_siblings)
                if len(h3_siblings_list):
                    column_name = data_source_div['data-source']
                    value_list = list(h3_siblings_list[1].strings)
                    column_value = ' '.join(x.strip() for x in value_list).strip()
                    if column_name == 'Attack':
                        attack_value = column_value
    
    return attack_value

In [15]:

unit_name_list = unit_availability_df.index.tolist()
for unit_name in unit_name_list:
    unit_soup = get_content_text_div(unit_name)
    if unit_soup is not None:
        attack_value = get_attack_from_aside(unit_soup)
        unit_availability_df.loc[unit_name, 'Attack'] = attack_value

https://ageofempires.fandom.com/wiki/Elite_Arambai got this error: HTTP Error 404: Not Found
https://ageofempires.fandom.com/wiki/Elite_Arambai got this error: object of type 'NoneType' has no len()
Elite Arambai got this error: 'NoneType' object has no attribute 'select'
https://ageofempires.fandom.com/wiki/Elite_Ballista_Elephant got this error: HTTP Error 404: Not Found
https://ageofempires.fandom.com/wiki/Elite_Ballista_Elephant got this error: object of type 'NoneType' has no len()
Elite Ballista Elephant got this error: 'NoneType' object has no attribute 'select'
https://ageofempires.fandom.com/wiki/Elite_Berserk got this error: HTTP Error 404: Not Found
https://ageofempires.fandom.com/wiki/Elite_Berserk got this error: object of type 'NoneType' has no len()
Elite Berserk got this error: 'NoneType' object has no attribute 'select'
https://ageofempires.fandom.com/wiki/Elite_Boyar got this error: HTTP Error 404: Not Found
https://ageofempires.fandom.com/wiki/Elite_Boyar got this er

https://ageofempires.fandom.com/wiki/Elite_Tarkan got this error: HTTP Error 404: Not Found
https://ageofempires.fandom.com/wiki/Elite_Tarkan got this error: object of type 'NoneType' has no len()
Elite Tarkan got this error: 'NoneType' object has no attribute 'select'
https://ageofempires.fandom.com/wiki/Elite_Teutonic_Knight got this error: HTTP Error 404: Not Found
https://ageofempires.fandom.com/wiki/Elite_Teutonic_Knight got this error: object of type 'NoneType' has no len()
Elite Teutonic Knight got this error: 'NoneType' object has no attribute 'select'
https://ageofempires.fandom.com/wiki/Elite_Throwing_Axeman got this error: HTTP Error 404: Not Found
https://ageofempires.fandom.com/wiki/Elite_Throwing_Axeman got this error: object of type 'NoneType' has no len()
Elite Throwing Axeman got this error: 'NoneType' object has no attribute 'select'
https://ageofempires.fandom.com/wiki/Elite_Turtle_Ship got this error: HTTP Error 404: Not Found
https://ageofempires.fandom.com/wiki/El

In [16]:

column_name = 'Attack'
unit_availability_df[column_name].unique()

array(['17 , 19  (Elite)  pierce', '6 pierce', '4 pierce',
       '8, 9 (Elite)  pierce', '2 melee', '12 melee',
       '9, 14 (Elite)  melee', '40 melee', '12, 14 (Elite)  melee',
       '6  melee', '7, 8 (Elite)  pierce', '35 melee', '3 melee',
       '6, 8 (Elite)  pierce', '9, 12 (Elite)  melee', '13 melee',
       '8 pierce 0 melee', '9  melee', '16, 18 (Elite)  pierce',
       '5 pierce', '90 melee', '110 melee', '4  melee', '7  melee',
       '6, 7 (Elite)  pierce', nan, '16 melee', '45 melee', '9 melee',
       '3 pierce', '3 pierce 1 melee', '1 pierce 0 melee',
       '2 pierce 1 melee', None, '8 pierce', '10, 13  (Elite)  melee',
       '6 melee', '17 pierce', '7 melee', '7 pierce', '140 melee',
       '16 pierce 0 melee', '10, 12 (Elite)  melee',
       '17 , 22  (Elite)  pierce', '7, 8 (Elite)  melee',
       '6 , 7  (Elite)  melee', '10 melee', '9, 10 (Elite)  melee',
       '8 , 10 (Elite)  melee', '4 melee', '50 melee',
       '16, 20 (Elite)  pierce', '14 melee', '25 me

In [27]:

regex_list = ['(\d+)', '(\d+) +\(\+(\d+) +upon +reaching +Feudal +Age *\) +melee',
              '(\d+) +melee', '(\d+) +pierce', '(\d+) +pierce +(\d+) +melee',
              '(\d+) *, +(\d+) +\(Elite\) +melee', '(\d+) *, +(\d+) +\(Elite\) +pierce']
attack_regex = re.compile('({})'.format('|'.join(regex_list)))
attack_regex.pattern

'((\\d+)|(\\d+) +\\(\\+(\\d+) +upon +reaching +Feudal +Age *\\) +melee|(\\d+) +melee|(\\d+) +pierce|(\\d+) +pierce +(\\d+) +melee|(\\d+) *, +(\\d+) +\\(Elite\\) +melee|(\\d+) *, +(\\d+) +\\(Elite\\) +pierce)'

In [17]:

# ['12 pierce 0 melee',
#  '3 (+2 upon reaching Feudal Age )  melee',
#  '16  melee']
regex_str = r'(\d+) +(\(\+(\d+) +upon +reaching +Feudal +Age *\) +)?'
regex_str += r'(pierce|melee)( +(\d+) +(pierce|melee))?'
feudal_regex = re.compile(regex_str)
def get_Attack_column(column_name):
    
    def f(attack):
        attack = str(attack)
        if re.fullmatch(r'[\d]+', attack):

            # Successful match at the start of the string
            return int(attack)

        else:
            match_obj = feudal_regex.search(attack)
            return_value = 0
            if match_obj:
                if match_obj.group(1) and match_obj.group(4):
                    first_value = int(match_obj.group(1))
                    if match_obj.group(3):
                        first_value = first_value + int(match_obj.group(3))
                    first_key = match_obj.group(4)
                    if column_name == first_key:
                        return_value = first_value
                    elif match_obj.group(6) and match_obj.group(7):
                        second_value = int(match_obj.group(6))
                        second_key = match_obj.group(7)
                        if column_name == second_key:
                            return_value = second_value

            return return_value

    new_column_name = 'Attack_{}_upon_reaching_Feudal_Age'.format(column_name)
    unit_availability_df[new_column_name] = unit_availability_df['Attack'].map(f)
    
    def f(attack):
        attack = str(attack)
        if re.fullmatch(r'[\d]+', attack):

            # Successful match at the start of the string
            return int(attack)

        else:
            match_obj = feudal_regex.search(attack)
            return_value = 0
            if match_obj:
                if match_obj.group(1) and match_obj.group(4):
                    first_value = int(match_obj.group(1))
                    first_key = match_obj.group(4)
                    if column_name == first_key:
                        return_value = first_value
                    elif match_obj.group(6) and match_obj.group(7):
                        second_value = int(match_obj.group(6))
                        second_key = match_obj.group(7)
                        if column_name == second_key:
                            return_value = second_value

            return return_value

    new_column_name = 'Attack_{}'.format(column_name)
    unit_availability_df[new_column_name] = unit_availability_df['Attack'].map(f)

for resource in ['pierce', 'melee']:
    get_Attack_column(resource)

In [18]:

columns_list = [cn for cn in unit_availability_df.columns if 'Attack' in cn]
unit_availability_df[columns_list].sample(5).T

Unit Name,Elite Turtle Ship,Heavy Cavalry Archer,Elite Plumed Archer,Cavalry Archer,Jaguar Warrior
AttackBonusvsArcher,0,0,0,0,0
AttackBonusvsBuilding,0,0,0,0,0
AttackBonusvsBuilding_unpacked,0,0,0,0,0
AttackBonusvsCamel,0,0,0,0,0
AttackBonusvsCastle,0,0,0,0,0
AttackBonusvsCavalry,0,0,0,0,0
AttackBonusvsCavalry_archer,0,0,0,0,0
AttackBonusvsCondottiero,0,0,2,0,10
AttackBonusvsEagle_Warrior,0,0,0,0,2
AttackBonusvsGunpowder_unit,0,0,0,0,0



# Clean up age and upgrades columns

In [ ]:

for cn in ['Age', 'Upgrades to', 'Upgraded from']:
    unit_availability_df[cn] = unit_availability_df[cn].fillna(value='')
    unit_availability_df[cn] = unit_availability_df[cn].map(lambda x: str(x).strip())

In [ ]:

columns_list = unit_availability_df.columns
file_path = os.path.join(DATA_CSV_FOLDER, 'unit_availability_df.csv')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df[columns_list].to_csv(file_path)
save_dataframes(include_index=True, unit_availability_df=unit_availability_df[columns_list])
xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'unit_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df[columns_list].to_excel(file_path)

In [ ]:

aside_columns_dict = load_object('aside_columns_dict')
aside_columns_dict

In [ ]:

match_series = pd.Series(cn in aside_columns_dict for cn in unit_availability_df.columns)
unit_availability_df.columns[match_series]

In [ ]:

cl = unit_availability_df.columns
acd = aside_columns_dict.copy()
unit_availability_df.columns = [cn if (cn not in acd) else acd[cn] for cn in cl]

In [ ]:

unit_availability_df['Age'].unique()

In [ ]:

unit_availability_df['MinRange'].unique()

In [ ]:

unit_availability_df['MinRange'] = unit_availability_df['MinRange'].fillna(0)

In [ ]:

unit_availability_df['Building'].unique()

In [ ]:

unit_availability_df['Building'] = unit_availability_df['Building'].map(lambda x: x.strip())

In [ ]:

unit_availability_df['Building'].unique()

In [ ]:

columns_list = unit_availability_df.columns
file_path = os.path.join(DATA_CSV_FOLDER, 'unit_availability_df.csv')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df[columns_list].to_csv(file_path)
save_dataframes(include_index=True, unit_availability_df=unit_availability_df[columns_list])
xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'unit_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df[columns_list].to_excel(file_path)

In [ ]:

def get_mixed(x):
    x = str(x)
    if re.search(r'([+.\d]+\s+[^+.\d]+|[^+.\d]+\s+[+.\d]+)', x):
        
        return True
    
    else:
        
        return False

digit_df = unit_availability_df.applymap(get_mixed)
for column_name in digit_df.columns[digit_df.any(axis=0)]:
    print(column_name)
    print(unit_availability_df[column_name].unique().tolist())

In [ ]:

unit_availability_df['UpTo'] = unit_availability_df['UpTo'].fillna(value='')
unit_availability_df['UpTo'] = unit_availability_df['UpTo'].map(lambda x: x.strip())
print(unit_availability_df['UpTo'].unique().tolist())

In [ ]:

rows_list = []
index_list = []
for row_index, row_series in unit_availability_df.iterrows():
    up_to = row_series['UpTo']
    if len(up_to) and (up_to not in unit_availability_df.index):
        row_series['UpFrom'] = row_series.name
        row_series['UpTo'] = ''
        row_series.name = up_to
        index_list.append(up_to)
        rows_list.append(row_series)
elite_df = pd.DataFrame(data=rows_list, index=index_list, columns=unit_availability_df.columns)
columns_list = ['UpFrom', 'UpTo']
elite_df[columns_list].head(5).T

In [ ]:

if elite_df.shape[0] > 0:
    unit_availability_df = pd.concat([unit_availability_df, elite_df])

In [ ]:

columns_list = unit_availability_df.columns
file_path = os.path.join(DATA_CSV_FOLDER, 'unit_availability_df.csv')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df[columns_list].to_csv(file_path)
save_dataframes(include_index=True, unit_availability_df=unit_availability_df[columns_list])
xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'unit_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df[columns_list].to_excel(file_path)

In [ ]:

x = '2.49'
match_list = re.findall(r'[.\d]+', str(x))
float(match_list[min(1, len(match_list)-1)])

In [ ]:

column_name = 'Range'
new_column_name = 'attack_or_conversion_vs_units'
def f(x):
    match_list = re.findall(r'[.\d]+', str(x))
    
    return float(match_list[0])

unit_availability_df[new_column_name] = unit_availability_df[column_name].map(f)

In [ ]:

column_name = 'Speed'
unit_availability_df[column_name].unique()

In [ ]:

unit_availability_df[column_name] = unit_availability_df[column_name].fillna(value=0)
match_series = unit_availability_df[column_name].isnull()
unit_availability_df[match_series].index

In [ ]:

match_series = unit_availability_df[column_name].isnull()
unit_availability_df[match_series].index

In [ ]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
save_dataframes(include_index=True, unit_availability_df=unit_availability_df)
file_path = os.path.join(xlsx_folder, 'unit_availability_df.xlsx')
unit_availability_df.to_excel(file_path)


...doing some manual cleanup in Excel...

In [ ]:

dataframes_dict = load_dataframes(unit_availability_df='unit_availability_df')
unit_availability_df = dataframes_dict['unit_availability_df']
unit_availability_df = unit_availability_df.set_index('Unit Name')
unit_availability_df.index.name = 'Unit Name'

In [ ]:

match_series = (unit_availability_df[column_name] == '1.2 (+0.35 upon reaching Feudal Age )')
unit_availability_df[match_series].index

In [ ]:

# ['1.3', '0.96', '0.8', '0.5', '0.85', '1.05', '0.7', '1.4', '1.45',
#  '1.1', '1.43', '1.35', '0.9', '1.2', '1.5', '1.6', '1.15', '1.25',
#  '1', '1.54', '0.6', '1.38', '1.26', '0.65',
#  '1.2 (+0.35 upon reaching Feudal Age )', '1.32', '0']
feudal_regex = re.compile(r'([.\d+]+) +\( *\+([.\d+]+) +upon +reaching +Feudal +Age *\)')
def f(row_series):
    speed = str(row_series['Speed'])
    if re.fullmatch(r'[.\d+]+', speed):
        
        # Successful match at the start of the string
        return float(speed)
    
    else:
        match_obj = feudal_regex.search(speed)
        if match_obj:
            speed = float(match_obj.group(1))
            bonus = float(match_obj.group(2))
            
            return speed + speed*bonus
        
        else:
            
            # Match attempt failed
            return float(speed)

unit_availability_df['Speed_upon_reaching_Feudal_Age'] = unit_availability_df.apply(f, axis=1)

In [ ]:

# ['1.3', '0.96', '0.8', '0.5', '0.85', '1.05', '0.7', '1.4', '1.45',
#  '1.1', '1.43', '1.35', '0.9', '1.2', '1.5', '1.6', '1.15', '1.25',
#  '1', '1.54', '0.6', '1.38', '1.26', '0.65',
#  '1.2 (+0.35 upon reaching Feudal Age )', '1.32', '0']
feudal_regex = re.compile(r'([.\d+]+) +\( *\+([.\d+]+) +upon +reaching +Feudal +Age *\)')
def f(row_series):
    speed = str(row_series['Speed'])
    if re.fullmatch(r'[.\d+]+', speed):
        
        # Successful match at the start of the string
        return float(speed)
    
    else:
        match_obj = feudal_regex.search(speed)
        if match_obj:
            speed = float(match_obj.group(1))
            bonus = float(match_obj.group(2))
            
            return float(speed)
        
        else:
            
            # Match attempt failed
            return float(speed)

unit_availability_df['Speed'] = unit_availability_df.apply(f, axis=1)

In [ ]:

unit_availability_df.columns

In [ ]:

column_name = 'TrainTime'
unit_availability_df[column_name].unique()

In [ ]:

f = lambda x: int(re.sub(' +seconds', '', x))
unit_availability_df['TrainTime'] = unit_availability_df['TrainTime'].map(f)

In [ ]:

import re

def get_mixed(x):
    x = str(x)
    if re.search(r'([+.\d]+\s+[^+.\d]+|[^+.\d]+\s+[+.\d]+)', x):
        
        return True
    
    else:
        
        return False

digit_df = unit_availability_df.applymap(get_mixed)
digit_df.columns[digit_df.any(axis=0)]

In [ ]:

column_name = 'UpCost'
unit_availability_df[column_name].unique()

In [ ]:

upcost_regex = re.compile(r' *(\d[,\d]+) +(food|gold|wood)( *, +(\d[,\d]+) +(food|gold|wood))?')
def get_UpCost_column(column_name):
    
    def f(up_cost):
        up_cost = str(up_cost)
        match_obj = upcost_regex.search(up_cost)
        return_value = '0'
        if match_obj:
            first_amount = match_obj.group(2)
            if column_name == first_amount:
                return_value = match_obj.group(1)
            else:
                if match_obj.group(3) is not None:
                    second_amount = match_obj.group(5)
                    if column_name == second_amount:
                        return_value = match_obj.group(4)
        return_value = int(re.sub(',', '', return_value))
        
        return return_value
    
    new_column_name = 'UpCost_{}'.format(column_name)
    unit_availability_df[new_column_name] = unit_availability_df['UpCost'].map(f)

for resource in ['food', 'gold', 'wood']:
    get_UpCost_column(resource)

In [ ]:

columns_list = ['UpCost_food', 'UpCost_gold', 'UpCost_wood']
unit_availability_df[columns_list].sample(5)

In [ ]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
save_dataframes(include_index=True, unit_availability_df=unit_availability_df)
file_path = os.path.join(xlsx_folder, 'unit_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df.to_excel(file_path)

In [ ]:

column_name = 'LOS'
unit_availability_df[column_name].unique()

In [ ]:

# ['5', '7', '6', '3', '4', '14', '15', '9', '8', '17', '10',
#  '8 (+2 upon reaching Imperial Age )', '11', '4 6 8 10', '19']
regex_str_list = ['([.\d+]+) +\( *\+([.\d+]+) +upon +reaching +Imperial +Age *\)',
                  '(\d+) +(\d+) +(\d+) +(\d+)']
imperial_regex = re.compile(r'|'.join(regex_str_list))
def f(row_series):
    los = str(row_series['LOS'])
    if re.fullmatch(r'[\d]+', los):
        
        # Successful match at the start of the string
        return int(los)
    
    else:
        match_obj = imperial_regex.search(los)
        if match_obj:
            if match_obj.group(1) and match_obj.group(2):
                los = int(match_obj.group(1))
                bonus = int(match_obj.group(2))
            
                return los + bonus
            
            elif match_obj.group(6):
                los = int(match_obj.group(6))
                
                return los
            
            else:

                # Match attempt failed
                return 0
            
        else:
            
            # Match attempt failed
            return 0

unit_availability_df['LOS_upon_reaching_Imperial_Age'] = unit_availability_df.apply(f, axis=1)

def f(row_series):
    los = str(row_series['LOS'])
    if re.fullmatch(r'[\d]+', los):
        
        # Successful match at the start of the string
        return int(los)
    
    else:
        match_obj = imperial_regex.search(los)
        if match_obj:
            if match_obj.group(1):
                los = int(match_obj.group(1))
            
                return los
            
            elif match_obj.group(5):
                los = int(match_obj.group(5))
                
                return los
            
            else:

                # Match attempt failed
                return 0
            
        else:
            
            # Match attempt failed
            return 0

unit_availability_df['LOS_upon_reaching_Castle_Age'] = unit_availability_df.apply(f, axis=1)

def f(row_series):
    los = str(row_series['LOS'])
    if re.fullmatch(r'[\d]+', los):
        
        # Successful match at the start of the string
        return int(los)
    
    else:
        match_obj = imperial_regex.search(los)
        if match_obj:
            if match_obj.group(1):
                los = int(match_obj.group(1))
            
                return los
            
            elif match_obj.group(4):
                los = int(match_obj.group(4))
                
                return los
            
            else:

                # Match attempt failed
                return 0
            
        else:
            
            # Match attempt failed
            return 0

unit_availability_df['LOS_upon_reaching_Feudal_Age'] = unit_availability_df.apply(f, axis=1)

def f(row_series):
    los = str(row_series['LOS'])
    if re.fullmatch(r'[\d]+', los):
        
        # Successful match at the start of the string
        return int(los)
    
    else:
        match_obj = imperial_regex.search(los)
        if match_obj:
            if match_obj.group(1):
                los = int(match_obj.group(1))
            
                return los
            
            elif match_obj.group(3):
                los = int(match_obj.group(3))
                
                return los
            
            else:

                # Match attempt failed
                return 0
            
        else:
            
            # Match attempt failed
            return 0

unit_availability_df['LOS_upon_reaching_Dark_Age'] = unit_availability_df.apply(f, axis=1)

In [ ]:

columns_list = ['LOS_upon_reaching_Imperial_Age', 'LOS_upon_reaching_Castle_Age',
                'LOS_upon_reaching_Feudal_Age', 'LOS_upon_reaching_Dark_Age']
unit_availability_df[columns_list].sample(5).T

In [ ]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
save_dataframes(include_index=True, unit_availability_df=unit_availability_df)
file_path = os.path.join(xlsx_folder, 'unit_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df.to_excel(file_path)

In [ ]:

column_name = 'Class'
unit_availability_df[column_name].unique()

In [ ]:

class_list = ['Archer', 'Camel', 'Condottiero', 'Eagle Warrior', 'Monk', 'Spearman',
              'Turtle Ship', 'Cavalry archer', 'Cavalry', 'Unique unit', 'War elephant',
              'Siege weapon', 'Ram', 'Ship', 'Gunpowder unit', 'Infantry']
class_regex = re.compile(r'({})'.format('|'.join(class_list)))
def get_Class_column(column_name):
    
    def f(column_value):
        column_value = str(column_value)
        return_value = 0
        for match_obj in class_regex.finditer(column_value):
            class_name = match_obj.group(1)
            if class_name == column_name:
                return_value = 1
        
        return return_value

    new_column_name = 'In_{}_Armor_class'.format(re.sub(' +', '_', column_name))
    unit_availability_df[new_column_name] = unit_availability_df['Class'].map(f)

for resource in class_list:
    get_Class_column(resource)

In [ ]:

columns_list = [cn for cn in unit_availability_df.columns if 'class' in cn.lower()]
unit_availability_df[columns_list].sample(5).T

In [ ]:

dataframes_dict = load_dataframes(unit_availability_df='unit_availability_df')
unit_availability_df = dataframes_dict['unit_availability_df']
unit_availability_df = unit_availability_df.set_index('Unit Name')
unit_availability_df.index.name = 'Unit Name'

In [ ]:

column_name = 'Gold'
unit_availability_df[column_name] = unit_availability_df[column_name].fillna(value=0)
unit_availability_df[column_name].unique()

In [ ]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
save_dataframes(include_index=True, unit_availability_df=unit_availability_df)
file_path = os.path.join(xlsx_folder, 'unit_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df.to_excel(file_path)

In [ ]:

import re

def get_mixed(x):
    x = str(x)
    if re.search(r'([+.\d]+\s+[^+.\d]+|[^+.\d]+\s+[+.\d]+)', x):
        
        return True
    
    else:
        
        return False

digit_df = unit_availability_df.applymap(get_mixed)
digit_df.columns[digit_df.any(axis=0)]

In [ ]:

column_name = 'Accuracy'
unit_availability_df[column_name].unique()

In [ ]:
# ['20%', '90%', '80%', '100%', nan, '92%', '95%', '50%', '85%',
#  '65%', '30%', '70%', '0%',
#  '80% against buildings 15% against units']
regex_str = r'(\d+)%( +against +buildings +(\d+)% +against +units)?'
accuracy_regex = re.compile(regex_str)
def f(accuracy):
    accuracy = str(accuracy)
    match_obj = accuracy_regex.search(accuracy)
    return_value = 0
    if match_obj:
        if match_obj.group(1):
            return_value = int(match_obj.group(1))

    return return_value

new_column_name = 'Accuracy_{}_against_buildings'.format(column_name)
unit_availability_df[new_column_name] = unit_availability_df['Accuracy'].map(f)

def f(accuracy):
    accuracy = str(accuracy)
    match_obj = accuracy_regex.search(accuracy)
    return_value = 0
    if match_obj:
        if match_obj.group(3):
            return_value = int(match_obj.group(3))
        elif match_obj.group(1):
            return_value = int(match_obj.group(1))

    return return_value

new_column_name = 'Accuracy_{}_against_units'.format(column_name)
unit_availability_df[new_column_name] = unit_availability_df['Accuracy'].map(f)

In [ ]:

columns_list = [cn for cn in unit_availability_df.columns if column_name in cn]
unit_availability_df[columns_list].sample(5).T

In [ ]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
save_dataframes(include_index=True, unit_availability_df=unit_availability_df)
file_path = os.path.join(xlsx_folder, 'unit_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df.to_excel(file_path)

In [ ]:

column_name = 'Type'
unit_availability_df[column_name].unique()

In [ ]:

import re

type_list = ['Cavalry Archer', 'Archer', 'Monk', 'Cavalry', 'Siege unit', 'Infantry',
             'Gunpowder unit', 'Naval vessel', 'Suicide unit', 'Siege weapon', 'Civilian Ship',
             'Healer', 'Civilian']
type_regex = re.compile(r'({})'.format('|'.join(type_list)))
def get_Type_column(column_name):
    
    def f(column_value):
        column_value = str(column_value)
        return_value = 0
        for match_obj in type_regex.finditer(column_value):
            type_name = match_obj.group(1)
            if type_name == column_name:
                return_value = 1
        
        return return_value

    new_column_name = 'In_{}_Unit_Type'.format(re.sub(' +', '_', column_name))
    unit_availability_df[new_column_name] = unit_availability_df['Type'].map(f)

for resource in type_list:
    get_Type_column(resource)

In [ ]:

[cn for cn in unit_availability_df.columns if 'type' in cn.lower()]

In [ ]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
save_dataframes(include_index=True, unit_availability_df=unit_availability_df)
file_path = os.path.join(xlsx_folder, 'unit_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
unit_availability_df.to_excel(file_path)

In [ ]:

column_descriptions_df = get_column_descriptions(unit_availability_df)
match_series = column_descriptions_df['dtype'].isin(['int64', 'float64'])
numeric_columns_list = column_descriptions_df[match_series]['column_name'].tolist()
non_columns_list = [cn for cn in unit_availability_df.columns if cn not in numeric_columns_list]
unit_availability_df[non_columns_list].sample(5).T

In [ ]:

description_columns_list = ['Intro', 'Heading', 'Required', 'Age', 'UpFrom', 'Building', 'UpTo']
unit_availability_df[description_columns_list].sample(5).T

In [ ]:

columns_list = description_columns_list + numeric_columns_list
unit_availability_df = unit_availability_df[columns_list].copy()
save_dataframes(include_index=True, unit_availability_df=unit_availability_df)

In [ ]:

unit_availability_df['Building'] = unit_availability_df['Building'].map(lambda x: str(x).strip())
match_series = (unit_availability_df['Building'] == 'Barracks')
unit_availability_df[match_series].index

In [ ]:

[cn for cn in unit_availability_df.columns if 'eleph' in cn.lower()]

In [ ]:

[cn for cn in unit_availability_df.index if 'eleph' in cn.lower()]

In [ ]:

dataframes_dict = load_dataframes(civilization_bonuses_df='civilization_bonuses_df')
civilization_bonuses_df = dataframes_dict['civilization_bonuses_df']
if 'Unnamed: 0' in civilization_bonuses_df.columns:
    civilization_bonuses_df = civilization_bonuses_df.set_index('Unnamed: 0')
elif 'Bonus Name' in civilization_bonuses_df.columns:
    civilization_bonuses_df = civilization_bonuses_df.set_index('Bonus Name')
civilization_bonuses_df.index.name = 'Bonus Name'

In [ ]:

civilization_bonuses_df.sample(5).T.sample(5).T

In [ ]:

# Battle Elephants move +15% faster.
civilization_bonuses_df = civilization_bonuses_df.T
#match_series = (unit_availability_df['Building'] == 'Barracks')
unit_list = ['Battle Elephant', 'Elite Battle Elephant']
for unit_name in unit_list:#unit_availability_df[match_series].index:
    for res_name in ['Speed']:
        column_name = '{}_{}'.format(re.sub(' +', '_', unit_name), res_name.lower())
        column_value = unit_availability_df.loc[unit_name, res_name]
        civilization_bonuses_df[column_name] = column_value

In [ ]:

civilization_bonuses_df = civilization_bonuses_df.T
civilization_bonuses_df.sample(5).T.sample(5).T

In [ ]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
save_dataframes(include_index=True, civilization_bonuses_df=civilization_bonuses_df)
file_path = os.path.join(xlsx_folder, 'civilization_bonuses_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
civilization_bonuses_df.to_excel(file_path)

In [ ]:

dataframes_dict = load_dataframes(technology_df='technology_df')
technology_df = dataframes_dict['technology_df']
technology_df = technology_df.set_index('Technology')
technology_df.index.name = 'Technology'

In [ ]:

# Blacksmith upgrades cost no gold.
match_series = (technology_df['Building'] == 'Blacksmith')
columns_list = ['Cost', 'Wood', 'Stone', 'Gold', 'Food']
for cn in columns_list[1:-1]:
    technology_df[cn] = technology_df[cn].fillna(0)
technology_df[match_series][columns_list]

In [ ]:

dataframes_dict = load_dataframes(civilization_bonuses_df='civilization_bonuses_df')
index_name = 'Bonus Name'
civilization_bonuses_df = dataframes_dict['civilization_bonuses_df']
if 'Unnamed: 0' in civilization_bonuses_df.columns:
    civilization_bonuses_df = civilization_bonuses_df.set_index('Unnamed: 0')
elif index_name in civilization_bonuses_df.columns:
    civilization_bonuses_df = civilization_bonuses_df.set_index(index_name)
civilization_bonuses_df.index.name = index_name

In [ ]:

dataframes_dict = load_dataframes(building_availability_df='building_availability_df')
index_name = 'Building Name'
building_availability_df = dataframes_dict['building_availability_df']
if 'Unnamed: 0' in building_availability_df.columns:
    building_availability_df = building_availability_df.set_index('Unnamed: 0')
elif index_name in building_availability_df.columns:
    building_availability_df = building_availability_df.set_index(index_name)
building_availability_df.index.name = index_name

In [ ]:

index_name = 'Unit Name'
dataframes_dict = load_dataframes(unit_availability_df='unit_availability_df')
unit_availability_df = dataframes_dict['unit_availability_df']
if 'Unnamed: 0' in unit_availability_df.columns:
    unit_availability_df = unit_availability_df.set_index('Unnamed: 0')
elif index_name in unit_availability_df.columns:
    unit_availability_df = unit_availability_df.set_index(index_name)
unit_availability_df.index.name = index_name

In [ ]:

# Cavalry has +20% HP.
resource_set = set()
for column_name in unit_availability_df.columns:
    if ('cavalry' in column_name.lower()) or ('hp' in column_name.lower()):
        resource_set.add(column_name)
resource_set

In [ ]:

# Cavalry has +20% HP.
match_series = (unit_availability_df['In_Cavalry_Armor_class'] == 1)
match_series = match_series | (unit_availability_df['In_Cavalry_Unit_Type'] == 1)
match_series = match_series | (unit_availability_df['In_Cavalry_Archer_Unit_Type'] == 1)
match_series = match_series | (unit_availability_df['In_Cavalry_archer_Armor_class'] == 1)
match_series = match_series & (unit_availability_df['Franks'] == 1)
columns_list = ['In_Cavalry_Armor_class', 'In_Cavalry_archer_Armor_class',
                'In_Cavalry_Unit_Type', 'In_Cavalry_Archer_Unit_Type']
unit_availability_df[match_series][columns_list].sort_values(columns_list)

In [ ]:

# Cavalry has +20% HP.
unit_set = set()
for index_name in unit_availability_df.index:
    if ('hp' in index_name.lower()) or ('cavalry' in index_name.lower()):
        unit_set.add(index_name)
unit_set

In [ ]:

# Cavalry has +20% HP.
unit_availability_df.loc[unit_set, resource_set].T

In [ ]:

import re

# Cavalry archers fire 25% faster.
civilization_bonuses_df = civilization_bonuses_df.T
column_name_list = []
for unit_name in unit_set:
    column_name = '{}_firing_rate'.format(re.sub(' +', '_', unit_name))
    column_value = unit_availability_df.loc[unit_name, 'PSpeed']
    column_name_list.append(column_name)
    civilization_bonuses_df[column_name] = column_value

In [ ]:

civilization_bonuses_df = civilization_bonuses_df.T
civilization_bonuses_df.T[column_name_list].sample(5).T

In [ ]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
save_dataframes(include_index=True, civilization_bonuses_df=civilization_bonuses_df)
file_path = os.path.join(xlsx_folder, 'civilization_bonuses_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
civilization_bonuses_df.to_excel(file_path)